In [3]:
# pip install chromadb

In [39]:
import chromadb
from chromadb.config import Settings

# You can configure Chroma to save and load from your local machine. Data will be persisted automatically and loaded on start (if it exists).
client = chromadb.PersistentClient(path="emopti_shared/chroma_vdb", settings=Settings(allow_reset=True))

In [40]:
client.heartbeat()
client.reset() # Empties and completely resets the database. ⚠️ This is destructive and not reversible.


True

In [28]:
import pandas as pd
import time
from ast import literal_eval

datafile_path = "./data/deidentified/aiipem_deidentified_embeddings_batch_0_to_10k.csv.gz"

start_time= time.time()
df_copy = pd.read_csv(datafile_path)
print(f"time taken to read csv: {time.time()- start_time}")

df_copy_bkp = df_copy.iloc[0:10].copy()

start_time= time.time()
df_copy_bkp["embedding"] = df_copy_bkp.embedding.apply(literal_eval)
print(f"time taken for literal_eval: {time.time()- start_time}")

display(df_copy_bkp.head(2))
df_copy_bkp.shape

time taken to read csv: 13.630956649780273
time taken for literal_eval: 0.22320938110351562


/tmp/ipykernel_41644/664382437.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_copy_bkp["embedding"] = df_copy_bkp.embedding.apply(literal_eval)


,PT_Visit_ID_Hashed,PT_ID_Hashed,Pt_Arrival_Method,Pt_Complaint,Pt_Triage_Acuity,Pt_Gender,Pt_Race,Pt_Ethnicity,Pt_Prime_ICD10,Pt_Secondary_ICD10,...,Arrival_TimeOfDay,Age_Group,Pt_DOB_Year,Systolic_BP,Diastolic_BP,emopti_grouped_disposition,grouped_arrival_method,bp_group,combined,embedding
0,c1cf913b612ffce254b33e996aa49b0def330b033058a7...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Ambulance,FLANK PAIN,3.0,F,UNKNOWN,Choose not to disclose,O26.893,R10.2,...,12pm-6pm,20-40,1988.0,111.0,62.0,Left Against Medical Advice,ambulance,normal,Complaint: FLANK PAIN; Triage_Acuity: 3.0; Gen...,"[0.009560050442814827, 0.002757575362920761, 0..."
1,094a742f233f2699fe3125b34ee61421e3f5ae54e26ca2...,e1866701283c7c46f57bff8b5d23c1d44fbbfca0a076a1...,Car,DIZZINESS,3.0,F,UNKNOWN,Choose not to disclose,E86.0,D62,...,6pm-Midnight,20-40,1988.0,115.0,56.0,Discharged,private transport,normal,Complaint: DIZZINESS; Triage_Acuity: 3.0; Gend...,"[0.011276663281023502, 0.003159793559461832, 0..."


(10, 29)

In [41]:
collection = client.create_collection(name="emopti_search")
collection = client.get_collection(name="emopti_search")

In [44]:
collection.peek() # returns a list of the first 10 items in the collection
collection.count() # returns the number of items in the collection

10

In [ ]:
# collection = client.create_collection(
#         name="collection_name",
#         metadata={"hnsw:space": "cosine"} # l2 is the default
#     )

In [30]:
df_copy_bkp.head(2)
ids = df_copy_bkp['PT_Visit_ID_Hashed'].values.tolist()
embeddings = df_copy_bkp['embedding'].values.tolist()

In [43]:
collection.add(
    embeddings=embeddings,
    ids=ids
)

In [47]:
search_results = collection.query(
    query_embeddings=[embeddings[0]],
    n_results=10
)

In [50]:
search_results.keys()

dict_keys(['ids', 'distances', 'metadatas', 'embeddings', 'documents', 'uris', 'data'])

In [54]:
search_results['ids']

[['c1cf913b612ffce254b33e996aa49b0def330b033058a782cb0d7e5de368a121',
  'd688cd7ca9f8eb18c785645559fa0983d21220e4e2f170f675549c1f48bf03e8',
  'a4a09bd3e9029d968c10a2a1a4d9afa6c46da103c7a8ea543f7714d593f17101',
  '36a8f9e2e5ec3e1857677ee3f331e86c696bb609e556ea30df9df54bf578fd66',
  '9c293f4ac80d2002f18b9941552789723ed606180be043506f37d1518d08371c',
  'ac081e45ac7a7d22b5dce07e53f42f6bfddd4c4587ce14bd4dd3a81483333042',
  '572c285eeb1a0af0fc77aa7c1e93cd6732131bf3c9da8f7a555d9a7996520504',
  'd790d52b70a07c98a22d02d378f21b0c63d2bb5df3679191d5583e4985e36c6a',
  '094a742f233f2699fe3125b34ee61421e3f5ae54e26ca27a490b74887dbc5c24',
  'f752254be3760eb3164cf379f7fd29203990f6be3f493c7982e43b3fb4428b0a']]

In [52]:
search_results['distances']

[[0.0,
  0.013424319598557109,
  0.017100975896066826,
  0.018255842803678972,
  0.10709413745519954,
  0.11221140582404583,
  0.11397947682826198,
  0.11800902929476231,
  0.15826637203662888,
  0.16155678308740726]]